In [1]:
!pip install python-dotenv
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 773.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 933.3 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 778.1 kB/s eta 0:00:000:00:01


In [2]:
from dotenv import load_dotenv
import os
import pyspark
from pyspark.sql import SparkSession
import requests
import json
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
import time

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.5.jar -P ./

--2025-11-03 01:31:06--  https://jdbc.postgresql.org/download/postgresql-42.2.5.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 825943 (807K) [application/java-archive]
Saving to: ‘./postgresql-42.2.5.jar.3’

postgresql-42.2.5.j 100%[===================>] 806.58K  1.08MB/s    in 0.7s    

2025-11-03 01:31:07 (1.08 MB/s) - ‘./postgresql-42.2.5.jar.3’ saved [825943/825943]



In [3]:
import os
import requests

jar_path = "/home/jovyan/work/postgresql-42.2.5.jar"
print(f"JAR file exists: {os.path.exists(jar_path)}")
print(f"JAR file size: {os.path.getsize(jar_path) if os.path.exists(jar_path) else 'N/A'} bytes")

from dotenv import load_dotenv
load_dotenv()

print(f"PORT_POSTGRES: {os.getenv('PORT_POSTGRES')}")
print(f"POSTGRES_DB: {os.getenv('POSTGRES_DB')}")
print(f"POSTGRES_USER: {os.getenv('POSTGRES_USER')}")
print(f"POSTGRES_PASSWORD set: {bool(os.getenv('POSTGRES_PASSWORD'))}")

JAR file exists: True
JAR file size: 825943 bytes
PORT_POSTGRES: 5432
POSTGRES_DB: ny_taxi
POSTGRES_USER: usuario_spark
POSTGRES_PASSWORD set: True


In [4]:
try:
    spark.stop()
except:
    pass

import pyspark 
from pyspark.sql import SparkSession 
from pyspark.sql import Row 

spark = SparkSession.builder.config("spark.jars", "/home/jovyan/work/postgresql-42.2.5.jar").master("local").appName("PySpark_Postgres_test").getOrCreate()

In [5]:
df = spark.read.format("jdbc") \
    .option("url", f"jdbc:postgresql://warehouses:5432/{os.getenv('POSTGRES_DB')}") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "raw.taxi_zone_lookup") \
    .option("user", os.getenv('POSTGRES_USER')) \
    .option("password", os.getenv('POSTGRES_PASSWORD')) \
    .load()

In [6]:
df.printSchema()

root
 |-- locationid: integer (nullable = true)
 |-- borough: string (nullable = true)
 |-- zone: string (nullable = true)
 |-- service_zone: string (nullable = true)
 |-- ingested_at_utc: timestamp (nullable = true)



In [7]:
def ingestar_zones_a_raw():
    SOURCE_PATH = os.getenv("SOURCE_PATH")
    path_url = f"{SOURCE_PATH}/misc/taxi_zone_lookup.csv"
    local_path = f"/tmp/taxiZones.parquet"
    
    # Descargo el archivo en carpeta temporal para posteriormente leerlo
    try:
        r = requests.get(path_url, stream=True)
        if r.status_code == 200:
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=10000000):
                    f.write(chunk)
        else:
            print(f"Archivo no encontrado en {path_url} (status {r.status_code})")
            return None
    except Exception as e:
        print(f"Error descargando {path_url}: {e}")
        return None
    else:
        print(f"Archivo obtenido exitosamente de: {path_url}")
    
    # Leo el archivo en un df de Spark
    try:
        df = spark.read.csv(local_path, header="true")
    except Exception as e:
        print(f"No se pudo leer {local_path}: {e}")
        return None
    else:
        print(f"Archivo leido exitosamente por Spark: {local_path}")

    conteoFilas = df.count()
    print(f"Ingestando hacia Snowflake datos de Zonas de Taxis. Total de filas: {conteoFilas}")

    try:
        df.write.format("jdbc") \
            .option("url", f"jdbc:postgresql://warehouses:5432/{os.getenv('POSTGRES_DB')}") \
            .option("driver", "org.postgresql.Driver") \
            .option("dbtable", "raw.taxi_zone_lookup") \
            .option("user", os.getenv('POSTGRES_USER')) \
            .option("password", os.getenv('POSTGRES_PASSWORD')) \
            .mode("overwrite") \
            .save()
    except Exception as e2:
        print(f"Error con ingreso de datos: {e2}")
        return None
    else:
        print("Zonas de taxis exportadas correctamente a Raw de Snowflake")

    #Me aseguro de eliminar el archivo parquet temporal
    try:
        os.remove(local_path)
        print(f"Archivo parquet temporal removido: {local_path}")
    except OSError as e:
        print(f"No se pudo remover el archivo parquet temporal {local_path}: {e}")

    #Retorno datos para tabla de conteos de datos consumidos por run
    return {
        "count": conteoFilas,
    }

In [8]:
#Cargo Zonas de Taxis en Snowflake
zonasTaxisIngesta=ingestar_zones_a_raw()
print(zonasTaxisIngesta)

Archivo obtenido exitosamente de: https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Archivo leido exitosamente por Spark: /tmp/taxiZones.parquet
Ingestando hacia Snowflake datos de Zonas de Taxis. Total de filas: 265
Zonas de taxis exportadas correctamente a Raw de Snowflake
Archivo parquet temporal removido: /tmp/taxiZones.parquet
{'count': 265}


In [9]:
import datetime

#Hago la presente funcion para generar un identificador unico asociado a cada carga de datos para el RUN_ID 
def generar_run_id():
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [10]:
import os 
import requests 
from pyspark.sql.functions import lit, current_timestamp 
from pyspark.sql.types import TimestampType

def ingestar_parquet_a_raw(service: str, year: int, month: int):
    SOURCE_PATH = os.getenv("SOURCE_PATH")
    path_url = f"{SOURCE_PATH}/trip-data/{service}_tripdata_{year}-{month:02d}.parquet"
    local_path = f"/tmp/{service}_tripdata_{year}-{month:02d}.parquet"
    print(path_url)
    
    # Descargo el archivo Parquet en carpeta temporal para posteriormente leerlo
    try:
        r = requests.get(path_url, stream=True)
        if r.status_code == 200:
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=10000000):
                    f.write(chunk)
        else:
            print(f"Archivo no encontrado en {path_url} (status {r.status_code})")
            return None
    except Exception as e:
        print(f"Error descargando {path_url}: {e}")
        return None
    else:
        print(f"Archivo obtenido exitosamente de: {path_url}")
    
    # Leo el archivo parquet en un df de Spark
    try:
        df = spark.read.parquet(local_path)
    except Exception as e:
        print(f"No se pudo leer {local_path}: {e}")
        return None
    else:
        print(f"Archivo leido exitosamente por Spark: {local_path}")

    run_id = generar_run_id()

    # Elimino columna conflictiva que solo esta presente en unos pocos parquets
    if 'cbd_congestion_fee' in df.columns:
        df = df.drop('cbd_congestion_fee')

    # Añado los metadatos indicados en instrucciones de PSET
    df_meta = df.withColumn("run_id", lit(run_id)) \
                .withColumn("service_type", lit(service)) \
                .withColumn("source_year", lit(year)) \
                .withColumn("source_month", lit(month)) \
                .withColumn("ingested_at_utc", current_timestamp()) \
                .withColumn("source_path", lit(path_url))

    # Convierto tipos de fecha a Timestamp porque me estaba marcando error al enviar datos al Snowflake sin esta transformacion
    for field in df_meta.schema.fields:
        if field.dataType.typeName() == "timestamp_ntz":
            df_meta = df_meta.withColumn(field.name, df_meta[field.name].cast(TimestampType()))

    primary_keys = ["tpep_pickup_datetime","tpep_dropoff_datetime","PULocationID","DoLocationID","VendorID"]

    if (service=="green"):
        primary_keys = ["lpep_pickup_datetime","lpep_dropoff_datetime","PULocationID","DoLocationID","VendorID"]

    df_meta = df_meta.dropDuplicates(primary_keys)

    conteoFilas = df_meta.count()
    print(f"Ingestando hacia Snowflake {service} {year}-{month}. Total de filas: {conteoFilas}")

    try:
        df_meta.write \
        .format("jdbc") \
        .option("url", f"jdbc:postgresql://warehouses:5432/{os.getenv('POSTGRES_DB')}") \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", f"raw.{service}_taxi_trip") \
        .option("user", os.getenv('POSTGRES_USER')) \
        .option("password", os.getenv('POSTGRES_PASSWORD')) \
        .mode("append") \
        .save()
    except Exception as e2:
        print(f"Error con tabla temporal: {e2}")
        return None

    #Me aseguro de eliminar el archivo parquet temporal
    try:
        os.remove(local_path)
        print(f"Archivo parquet temporal removido: {local_path}")
    except OSError as e:
        print(f"No se pudo remover el archivo parquet temporal {local_path}: {e}")

    #Retorno datos para tabla de conteos de datos consumidos por run
    return {
        "year": year,
        "month": month,
        "count": conteoFilas,
        "run_id": run_id,
        "service_type": service
    }

In [ ]:
import json
import os
import requests

#Defino funciones tipicas de checkpoint para en caso de fallo no ingestar datos desde cero
def save_checkpoint(year, month):
    with open(CHECKPOINT_FILE, "w") as f:
        json.dump({"year": year, "month": month}, f)

def load_checkpoint(CHECKPOINT_FILE):
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r") as f:
            return json.load(f)
    return {"year": 0, "month": 0}

#Genero arreglo que contendra datos de ingesta para tabla de conteos
resultadosGeneralesIngesta=[]

try:
    #Leo los datos de tipos de taxis, years y months desde variables de entorno
    tipos_taxis=os.getenv("SERVICES").split(',')
    for tipo_taxi in tipos_taxis:
        
        lista_years = sorted([int(item) for item in (os.getenv("YEARS").split(','))])
        lista_months = sorted([int(item) for item in (os.getenv("MONTHS").split(','))])
        #Cargo el checkpoint y en caso de que tenga registros recorto los arreglos de months y years para recorrer desde ultima ingesta exitosa
        CHECKPOINT_FILE = f"checkpointTaxis{tipo_taxi.capitalize()}.json"
        print(CHECKPOINT_FILE)
        checkpoint=load_checkpoint(CHECKPOINT_FILE)
        print(f"checkpoint: {checkpoint}")
        
        if ( checkpoint != {"year": 0, "month": 0} and (int(checkpoint["month"]) in lista_months) and (int(checkpoint["year"]) in lista_years)):    
            if ( int(checkpoint["month"]) == lista_months[-1] and int(checkpoint["year"]) != lista_years[-1] ):
                lista_years= lista_years[lista_years.index(checkpoint["year"])+1:]
            elif ( int(checkpoint["month"]) == lista_months[-1] and int(checkpoint["year"]) == lista_years[-1] ): 
                continue
            else:
                lista_years= lista_years[lista_years.index(checkpoint["year"]):]
                lista_months= lista_months[lista_months.index(checkpoint["month"])+1:]
        
        for year_taxi in lista_years:
            for month_taxi in lista_months:  
                #Llamo a la funcion de ingesta de datos iterativamente para cada mes, year y tipo de taxi
                print(f"Iniciando ingesta de datos de taxis {tipo_taxi}: {month_taxi}-{year_taxi}")
                resultadosParciales=ingestar_parquet_a_raw(tipo_taxi, year_taxi, month_taxi)
                #Guardo los resultados y genero el checkpint
                if (resultadosParciales != None):
                    resultadosGeneralesIngesta.append(resultadosParciales)
                    save_checkpoint(year_taxi,month_taxi)
            lista_months = sorted([int(item) for item in (os.getenv("MONTHS").split(','))])
                
except Exception as e5:
    #Como en todas las funciones vistas hago manejo de errores
    print(f"Fallo el proceso de ingesta masiva de datos de taxis NY: {e5}")
else:
    print("El proceso de ingesta masiva de taxis NY fue exitoso")

checkpointTaxisYellow.json
checkpoint: {'year': 0, 'month': 0}
Iniciando ingesta de datos de taxis yellow: 1-2022
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Archivo obtenido exitosamente de: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Archivo leido exitosamente por Spark: /tmp/yellow_tripdata_2022-01.parquet
Ingestando hacia Snowflake yellow 2022-1. Total de filas: 2452562
Archivo parquet temporal removido: /tmp/yellow_tripdata_2022-01.parquet
Iniciando ingesta de datos de taxis yellow: 2-2022
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Archivo obtenido exitosamente de: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Archivo leido exitosamente por Spark: /tmp/yellow_tripdata_2022-02.parquet
Ingestando hacia Snowflake yellow 2022-2. Total de filas: 2965646
Archivo parquet temporal removido: /tmp/yellow_tripdata_2022-02.parquet
Iniciando ingesta de 